# Forecasting Methods 3: Prophet

## 3/28/25

In the last notebooks, we covered using ARIMA and AutoARIMA to forecast next-day weather. In this notebook, we will try out Meta's Prophet model.

Prophet is a machine learning model used to forecast time-series data that is well-equipped to handle seasonality and holiday effects. This model will be used in the project in conjunction with AutoARIMA to create a robust forecast.

## 1. Importing Packages and Data

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from backend.regionweather import RegionWeather

region_data = {
    'US-FLA-FMPP': {'lat': 28.525581, 'lon': -81.536775, 'alt': 0},
    'US-FLA-FPC': {'lat': 28.996695, 'lon': -82.886613, 'alt': 0},
    'US-FLA-FPL': {'lat': 27.917488, 'lon': -81.450970, 'alt': 0},
    'US-FLA-GVL': {'lat': 29.619310, 'lon': -82.328732, 'alt': 0},
    'US-FLA-HST': {'lat': 25.456904, 'lon': -80.588092, 'alt': 0},
    'US-FLA-JEA': {'lat': 30.390902, 'lon': -83.679837, 'alt': 0},
    'US-FLA-SEC': {'lat': 28.805983, 'lon': -82.306291, 'alt': 0},
    'US-FLA-TAL': {'lat': 30.437174, 'lon': -84.248042, 'alt': 0},
    'US-FLA-TEC': {'lat': 27.959413, 'lon': -82.144821, 'alt': 0}
}

In [9]:
region = 'US-FLA-FMPP'
region1 = region_data['US-FLA-FMPP']
end = dt.datetime.today()
start = end - dt.timedelta(days=365)

rw = RegionWeather(region, region1['lat'],region1['lon'],region1['alt'],start,end)

Fetching Hourly Object...
Hourly Object Fetched!
Fetching Hourly Data from Object...
Hourly Object Fetched!
Hourly Data Cleaned!
Fetching Daily Data...
Daily Data Fetched!
Fetching Weekly Data...
Weekly Data Aggregated and Fetched!
Fetching Monthly Data...
Fetching Daily Data...
Monthly Data Fetched!
15 Minute Data Interpolated!


In [10]:
rw_dict = rw.to_dict()

temp = rw_dict['df_15m']['temp'].reset_index().dropna()
temp.columns = ['ds','y']
# temp['unique_id'] = '1'
temp

,ds,y
0,2024-03-28 18:00:00,23.500
1,2024-03-28 18:15:00,24.075
2,2024-03-28 18:30:00,24.650
3,2024-03-28 18:45:00,25.225
4,2024-03-28 19:00:00,25.800
...,...,...
35032,2025-03-28 16:00:00,24.100
35033,2025-03-28 16:15:00,24.200
35034,2025-03-28 16:30:00,24.300
35035,2025-03-28 16:45:00,24.400


## 2. Prophet Quickstart Guide

This has to be edited to match 15-minute data. The following code creates a Prophet object, fits the model, makes a forecast and evaluates it.

In [13]:
from prophet import Prophet

model = Prophet(
    seasonality_mode='multiplicative',
    # Add daily seasonality since we have 15-min data
    daily_seasonality=True,
    # Add weekly seasonality
    weekly_seasonality=True,
    # Add yearly seasonality
    yearly_seasonality=True,
    # Add change point detection
    changepoint_prior_scale=0.05,
    # Add seasonality prior scale
    seasonality_prior_scale=10.0
)
    
model.fit(temp)

18:24:53 - cmdstanpy - INFO - Chain [1] start processing
18:25:14 - cmdstanpy - INFO - Chain [1] done processing


In [14]:
future = model.make_future_dataframe(
    periods=24*4, # 15 mins * 4 * 24 = 1 day of 15-min data
    freq='15min',
    include_history=True
)

forecast = model.predict(future)

In [17]:
forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,daily,daily_lower,daily_upper,multiplicative_terms,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2024-03-28 18:00:00,35.055074,23.748781,30.637755,35.055074,35.055074,0.168877,0.168877,0.168877,-0.223994,...,-0.009927,-0.009927,-0.009927,-0.382944,-0.382944,-0.382944,0.0,0.0,0.0,27.202937
1,2024-03-28 18:15:00,35.053569,23.802192,30.706209,35.053569,35.053569,0.170729,0.170729,0.170729,-0.222351,...,-0.010105,-0.010105,-0.010105,-0.382975,-0.382975,-0.382975,0.0,0.0,0.0,27.259360
2,2024-03-28 18:30:00,35.052065,23.945428,30.889997,35.052065,35.052065,0.171917,0.171917,0.171917,-0.221368,...,-0.010278,-0.010278,-0.010278,-0.383007,-0.383007,-0.383007,0.0,0.0,0.0,27.292658
3,2024-03-28 18:45:00,35.050560,23.954864,30.848184,35.050560,35.050560,0.172392,0.172392,0.172392,-0.221095,...,-0.010448,-0.010448,-0.010448,-0.383039,-0.383039,-0.383039,0.0,0.0,0.0,27.301071
4,2024-03-28 19:00:00,35.049056,23.725766,30.538825,35.049056,35.049056,0.172089,0.172089,0.172089,-0.221594,...,-0.010613,-0.010613,-0.010613,-0.383071,-0.383071,-0.383071,0.0,0.0,0.0,27.282390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35128,2025-03-29 16:00:00,32.543524,20.650385,27.456440,32.543524,32.543524,0.129978,0.129978,0.129978,-0.261645,...,-0.006764,-0.006764,-0.006764,-0.384859,-0.384859,-0.384859,0.0,0.0,0.0,24.028677
35129,2025-03-29 16:15:00,32.544832,20.854967,27.652825,32.544832,32.544832,0.137562,0.137562,0.137562,-0.253991,...,-0.006666,-0.006666,-0.006666,-0.384887,-0.384887,-0.384887,0.0,0.0,0.0,24.278736
35130,2025-03-29 16:30:00,32.546139,21.138575,27.961748,32.546139,32.546139,0.144245,0.144245,0.144245,-0.247236,...,-0.006566,-0.006566,-0.006566,-0.384915,-0.384915,-0.384915,0.0,0.0,0.0,24.499547
35131,2025-03-29 16:45:00,32.547447,21.157682,28.159068,32.547447,32.547447,0.150095,0.150095,0.150095,-0.241312,...,-0.006465,-0.006465,-0.006465,-0.384943,-0.384943,-0.384943,0.0,0.0,0.0,24.693359
